## Data cleaning & EDA

In [1]:
import re
import pandas as pd

In [ ]:
df_benign = pd.read_csv('../Data/CSV_benign.csv')
df_malware = pd.read_csv('../Data/CSV_malware.csv')

# 'Country' column name is duplicated in malware csv, therefore I decided to rename both. While reading it, pandas reads duplicated column name with '.1' suffix
df_benign.rename(columns={'Country.1':'Country_1'}, inplace=True)
df_malware.rename(columns={'Country.1':'Country_1'}, inplace=True)

# Reindex columns
df_benign = df_benign.reindex(sorted(df_benign.columns), axis=1)
df_malware = df_malware.reindex(sorted(df_malware.columns), axis=1)
# df_malware.head(3)

In [3]:
# A few basic data checks
print('Shape of benign df: ', df_benign.shape, ' | Shape of malware df: ', df_malware.shape)
print(f'The dataset contains a high imbalance: among {df_benign.shape[0]+df_malware.shape[0]} total samples (w/o dropping duplicates)')
print(f'only {df_malware.shape[0]} samples are marked as threat, which is {(df_malware.shape[0]/(df_benign.shape[0]+df_malware.shape[0]))*100:.3f}%  (w/o dropping duplicates)')
print(f'# of columns in df_benign: {df_benign.columns.value_counts().sum()} | # of columns in df_malware: {df_malware.columns.value_counts().sum()}')
print(f'Does two dataframes column names match? {df_benign.columns.tolist().sort() == df_malware.columns.tolist().sort()}')
print(f'Dtypes match check: \n{df_benign.dtypes == df_malware.dtypes}')

Shape of benign df:  (494135, 38)  | Shape of malware df:  (4999, 38)
The dataset contains a high imbalance: among 499134 total samples (w/o dropping duplicates)
only 4999 samples are marked as threat, which is 1.002%  (w/o dropping duplicates)
# of columns in df_benign: 38 | # of columns in df_malware: 38
Does two dataframes column names match? True
Dtypes match check: 
1gram                  True
2gram                  True
3gram                  True
ASN                    True
Alexa_Rank            False
Country                True
Country_1              True
Creation_Date_Time     True
Domain                 True
Domain_Age             True
Domain_Name            True
Emails                 True
IP                     True
Name_Server_Count     False
Organization           True
Page_Rank              True
Registrant_Name        True
Registrar              True
State                  True
TTL                   False
char_distribution      True
dec_32                False
dec_8     

In [ ]:
# By digging into missmatch in dtypes, I was able to identify 'mixing' of columns data in 24 records of df_malware.
# To fix it, following steps are taken: 1. Identify incorrect rows by checking len of IP column,
#   2. Get records into new df
#   3. Rename columns
#   4. Drop incorrect rows from df_malware
#   5. Concatenate fixed data to df_malware

incorrect_rows_idx = df_malware.index[df_malware['IP'].str.len()==2]
df_incorrect_rows = df_malware.iloc[incorrect_rows_idx]

# Applies to df_malware only - rename of columns for 24 records
col_val_replace_to = {
    'Country': 'TTL',
    'TTL': 'Domain',
    'IP': 'Country',
    'Domain': 'IP',
}

df_incorrect_rows.rename(columns=col_val_replace_to, inplace=True) # Apply rename
df_malware.drop(incorrect_rows_idx, axis=0, inplace=True) # Drop from malware df incorrect rows
df_malware = pd.concat([df_malware, df_incorrect_rows], ignore_index=False) # Concatenate fixed data

In [5]:
df_benign.index[df_benign['IP'].str.len()==2] # Proof: this case does not affect benign df

Index([], dtype='int64')

In [6]:
# Drop from malware df 2 records with domain equal to 397220 (incorrectly imputed data)
df_malware.drop(df_malware.index[df_malware['Domain']=='397220'], axis=0, inplace=True)

In [7]:
# Just to make sure, domains in benign df are also checked (by looking at csv) - after checking it domains look fine
unique_domains_benign = df_benign['Domain'].unique()
# pd.DataFrame(unique_domains_benign).to_csv('./tmp/unique_domains_benign.csv')

In [8]:
# Dataset contains a lot of null/nan in columns like e.g. Country, ASN, IP, Domain_Name, State and so on.
# It is a common knowledge that ASN and IP might change (e.g. by a load balancer), however acording to
# Mahdavifar et al. (2021) the data has been gathered in a sliding window τ, which was relatively short (approx. 0,7s for b'instagram.com.' domain in benign df).
# Therefore, I made an assumption that only [Domain_Age] might be different across samples of the same domain.
# In the code below I used forward and backward filling of missing values for certain domain, 
#   followed by work on nulls and drop of duplicates.
# Primary task is to classify malicious domains, therefore drop of redundant data is reasonable (note that it might be distincted by [Domain_Age]).

def custom_filling(df):
    if not df['Domain'].isnull().sum().sum() == 0:
        raise Exception("Exception: The dataset contains null values in Domain column! Please provide correct dataset or change filling approach.")
    
    print('Df shape before the filling process & dropping duplicates: ', df.shape)
    # df = df.groupby(['Domain']).apply(lambda group: group.ffill()) # TODO/Done: Improve performance + bfill

    df['Domain_tmp'] = df['Domain'] # The reason to do it this way is that as_index is ignored in groupby op, and it is anyway faster than apply approach
    df = df.groupby(['Domain_tmp'], as_index=False).ffill()

    df['Domain_tmp'] = df['Domain'] # The reason to do it this way is that as_index is ignored in groupby op, and it is anyway faster than apply approach
    df = df.groupby(['Domain_tmp'], as_index=False).bfill()
    df.drop(columns=['Domain_Age'], inplace=True)
    df.drop_duplicates(inplace=True)
    print('After filling & drop of duplicates: ', df.shape)
    print('Checking for nulls not filled: \n', df.isnull().sum(axis = 0))
    return df

df_malware = custom_filling(df_malware)
df_benign = custom_filling(df_benign)

Df shape before the filling process & dropping duplicates:  (4997, 38)
After filling & drop of duplicates:  (4205, 37)
Checking for nulls not filled: 
 1gram                    0
2gram                    0
3gram                    0
ASN                     55
Alexa_Rank             457
Country                 57
Country_1             1780
Creation_Date_Time     950
Domain                   0
Domain_Name            823
Emails                1342
IP                       9
Name_Server_Count      457
Organization          2310
Page_Rank              457
Registrant_Name       4139
Registrar             1066
State                 2010
TTL                      0
char_distribution        0
dec_32                   0
dec_8                    0
entropy                  0
hex_32                   0
hex_8                    0
len                      0
longest_word             0
numeric_percentage       0
obfuscate_at_sign        0
oc_32                    0
oc_8                     0
puny_coded 

In [9]:
# To mitigate missing values across similar columns like Domain, Domain_Name and Country, Country_1, following code is applied to df's
# The code also applies mapping to unify a bit entries
countries_map = {
    '-':'',
    "china":"CN",
    "Malaysia":'ID',
    "United States":"US",
    "TURKEY":'TR',
    'RUSSIA':'RU',
    'Russian Federation':'RU',
    'Belarus':'BY',
    'Korea':'KR',
}

def use_regex(input_text):
    return re.sub(r"b'(.+?).'", r"\1", input_text)

def impute_similar_cols(df):
    df["Country_1"].replace(countries_map, inplace=True)
    df["Country"].replace(countries_map, inplace=True)
    df["Country_1"].fillna(df["Country"], inplace=True)
    df["Country"].fillna(df["Country_1"], inplace=True)
    df["Domain_Name"].fillna(df["Domain"].apply(use_regex), inplace=True)
    return df

df_malware = impute_similar_cols(df_malware)
df_benign = impute_similar_cols(df_benign)

In [10]:
print(f'Dtypes match check: \n{df_benign.dtypes == df_malware.dtypes}')

Dtypes match check: 
1gram                  True
2gram                  True
3gram                  True
ASN                    True
Alexa_Rank            False
Country                True
Country_1              True
Creation_Date_Time     True
Domain                 True
Domain_Name            True
Emails                 True
IP                     True
Name_Server_Count     False
Organization           True
Page_Rank              True
Registrant_Name        True
Registrar              True
State                  True
TTL                   False
char_distribution      True
dec_32                False
dec_8                 False
entropy               False
hex_32                False
hex_8                 False
len                   False
longest_word           True
numeric_percentage    False
obfuscate_at_sign     False
oc_32                 False
oc_8                  False
puny_coded            False
shortened             False
sld                    True
subdomain             False

In [15]:
# Not all columns with missmatch data type are being processed in this function
def process_cols_to_type(df):
    df['Alexa_Rank'] = pd.to_numeric(df["Alexa_Rank"], errors='coerce') # Should be number
    df['Alexa_Rank'].fillna(0, inplace=True)
    df['Name_Server_Count'] = pd.to_numeric(df["Name_Server_Count"], errors='coerce') # Should be number
    df['TTL'] = pd.to_numeric(df["TTL"], errors='coerce') # Should be number
    df['entropy'] = pd.to_numeric(df["entropy"], errors='coerce') # Should be number
    df['len'] = pd.to_numeric(df["len"], errors='coerce') # Should be number - Length of domain and subdomain
    df['len'].fillna(len(df['Domain_Name']), inplace=True)
    df['numeric_percentage'] = pd.to_numeric(df["numeric_percentage"], errors='coerce') # Should be number - Counts the number of digits in domain and subdomain
    df['Registrant_Name'] = df['Registrant_Name'].astype('category')
    df['Organization'] = df['Organization'].astype('category')
    df['Country'] = df['Country'].astype('category')
    df['Emails'] = df['Emails'].astype('category')
    df['State'] = df['State'].astype('category')
    df['tld'] = df['tld'].astype('category')
    df['Registrar'] = df['Registrar'].astype('category')
    df['Domain_Name'] = df['Domain_Name'].astype('category')
    df['sld'] = df['sld'].astype('category')
    return df

# selected_cols = ['Registrant_Name','Organization','Country','Emails','State','tld','Registrar','Domain_Name','TTL','sld','entropy']
df_malware = process_cols_to_type(df_malware)
df_benign = process_cols_to_type(df_benign)

In [16]:
# # Store data to csv for manual analysis
# df_benign.to_csv('./tmp/df_benign_filled.csv')
# df_malware.to_csv('./tmp/df_malware_filled.csv')

In [17]:
df_benign = process_cols_to_type(df_benign)
df_malware = process_cols_to_type(df_malware)

# Select relevant columns
selected_cols = ['Alexa_Rank','Registrant_Name','Organization','Country','Emails','State','tld','Registrar','Domain_Name','TTL','sld','entropy']

df_benign = df_benign[selected_cols]
df_malware = df_malware[selected_cols]

In [19]:
df_benign['is_threat'] = 0
df_malware['is_threat'] = 1

valid_benign = df_benign.iloc[-100:].copy()
df_benign = df_benign.iloc[:-100]
valid_malware = df_malware.iloc[-100:].copy()
df_malware = df_malware.iloc[:-100]

In [20]:
# Combine two dataframes
df = pd.concat([df_benign, df_malware])
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

X = df.drop(columns='is_threat')
y = df['is_threat']

df_valid = pd.concat([valid_benign, valid_malware])
df_valid.drop_duplicates(inplace=True)
df_valid.reset_index(drop=True, inplace=True)

X_valid = df_valid.drop(columns='is_threat')
y_valid = df_valid['is_threat']

In [21]:
from imblearn.combine import SMOTETomek

def balance_data(X_part, y_part):
    X_bal, y_bal = SMOTETomek(sampling_strategy='auto').fit_resample(X_part, y_part)
    return X_bal, y_bal

# X, y = balance_data(X, y)

## Modelling

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

label_encoder = LabelEncoder()
scaler = MinMaxScaler()
clf = XGBClassifier()
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize variables to store evaluation metrics
accuracy_scores = []
balanced_acc_scores = []
f1_scores = []
roc_aucs = []

# Train and evaluate the classifier for each split
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for col in X_train.columns:
            if not pd.api.types.is_numeric_dtype(X_train[col]):
                X_train.loc[:, col] = label_encoder.fit_transform(X_train[col])

    for col in X_test.columns:
            if not pd.api.types.is_numeric_dtype(X_test[col]):
                X_test.loc[:, col] = label_encoder.fit_transform(X_test[col])

    scaler.fit(X_train.fillna(0))  # Fit on the training data
    X_train = pd.DataFrame(scaler.transform(X_train.fillna(0)), columns=X_train.columns)  # Transform the training data
    X_test = pd.DataFrame(scaler.transform(X_test.fillna(0)), columns=X_test.columns)    # Transform the test data

    # X_train, y_train = balance_data(X_train, y_train)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    y_prob = clf.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_prob)

    accuracy_scores.append(accuracy)
    balanced_acc_scores.append(balanced_acc)
    f1_scores.append(f1)
    roc_aucs.append(roc_auc)

# Calculate and print average metrics
avg_accuracy = np.mean(accuracy_scores)
avg_balanced_acc = np.mean(balanced_acc_scores)
avg_f1 = np.mean(f1_scores)
avg_roc_auc = np.mean(roc_aucs)

print(f"Average Accuracy: {avg_accuracy:.2f}")
print(f"Average Balanced Accuracy: {avg_balanced_acc:.2f}")
print(f"Average F1 Score: {avg_f1:.2f}")
print(f"Average ROC AUC: {avg_roc_auc:.2f}")

Average Accuracy: 0.99
Average Balanced Accuracy: 0.52
Average F1 Score: 0.98
Average ROC AUC: 0.83


In [23]:
for col in X_valid.columns:
    if not pd.api.types.is_numeric_dtype(X_valid[col]):
        X_valid.loc[:, col] = label_encoder.fit_transform(X_valid[col])

X_valid = pd.DataFrame(scaler.transform(X_valid.fillna(0)), columns=X_valid.columns)
y_val_pred = clf.predict(X_valid)

val_accuracy = accuracy_score(y_valid, y_val_pred)
val_balanced_acc = balanced_accuracy_score(y_valid, y_val_pred)
val_f1 = f1_score(y_valid, y_val_pred, average='weighted')
y_val_prob = clf.predict_proba(X_valid)[:, 1]
val_roc_auc = roc_auc_score(y_valid, y_val_prob)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation Balanced Accuracy: {val_balanced_acc:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")
print(f"Validation ROC AUC: {val_roc_auc:.2f}")

Validation Accuracy: 0.49
Validation Balanced Accuracy: 0.50
Validation F1 Score: 0.32
Validation ROC AUC: 0.65


In [24]:
y.value_counts()

is_threat
0    362878
1      4056
Name: count, dtype: int64

In [25]:
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    num_ones_in_y_test = (y_test == 1).sum()
    num_ones_in_y_train = (y_train == 1).sum()
    print('Number of 1s in y_test:', num_ones_in_y_test,' | ', 'Number of 1s in y_train:', num_ones_in_y_train)

Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245
Number of 1s in y_test: 811  |  Number of 1s in y_train: 3245


In [26]:
y.value_counts()

is_threat
0    362878
1      4056
Name: count, dtype: int64

## References

Samaneh Mahdavifar, Nasim Maleki, Arash Habibi Lashkari, Matt Broda, Amir H. Razavi, “Classifying Malicious Domains using DNS Traffic Analysis”, The 19th IEEE International Conference on Dependable, Autonomic, and Secure Computing (DASC), Oct. 25-28, 2021, Calgary, Canada